In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
import pandas as pd
from pyspark.sql.functions import row_number,lit,desc,monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,StringType,DateType,FloatType

In [ ]:
spark=SparkSession.builder.appName("Basics").getOrCreate()

In [ ]:
df=pd.read_excel('/content/IcecreamDataset.xlsx')

In [ ]:
df['ProductName'].unique()

array(['STRABERY CHEESE CAKE MOMENT', 'CHOC FUDGE BROWNI MOMENT',
       'KING CONE 2/1 (1X25)', 'MANGO CLASSIC CUP (1X16)',
       'CHEER VANILA CUP (1X24)', 'MANGO RIPPLE SUPREME',
       'MAMAMIA (1X50)', 'BLUE BERRY SUPREME', 'ROYAL CONE (1X25)',
       'STRAWBERRY PARTY PACK', 'SHAHI KULFA FAMILY PACK',
       'FUNDE STRABERRY CUP (1X16)', 'TURTLE (1X30)', 'MANGO FAMILY PACK',
       'PISTA NUTS BULK PACK', 'PISTACHIO PARTY PACK',
       'CHEER CUP STRAW LASSI (1X24)', 'CHECKER (1X40)',
       'FUNDE CHOCLATE CUP (1X16)', 'CHOC CHIP SUPREME',
       'MANGO PARTY PACK', 'MOMENT CONE (1X25)', 'BALL CONE (1X20)',
       'BADSHAHI KULFI (1X45)', 'SHAHI KULFA CLASSIC CUP (1X16)',
       'SHAHI KULFA SOFT N COOL', 'TRIPPLE DELIGHT',
       'VANILLA FUDGE SUPREME', 'PISTACHO FAMILY PACK', 'TROPICA (1X50)',
       'SHAHI KULFA PARTY PACK', 'CHOC BAR (1X45)',
       'STRAW RIPPLE SUPREME', 'ROSE PARTY PACK', 'VANILLA BULK PACK',
       'PISTA NUTS SOFT N COOL', 'ICE CREAM SODA (1x50)',
   

In [ ]:
print(df.head())

   SalesDate  SalesQty  SalesAmount ProductCategory ProductSubCategory  \
0 2022-09-02         2      36956.0      ICE CREAMS         MOMENT TUB   
1 2022-09-05         5      92390.0      ICE CREAMS         MOMENT TUB   
2 2022-01-24        25      40325.0           CONES        2 IN 1 CONE   
3 2022-01-15        16      25808.0            CUPS        CLASSIC CUP   
4 2022-02-11        12      13272.0            CUPS          CHEER CUP   

                   ProductName StoreProvince StoreRegion    StoreZone  \
0  STRABERY CHEESE CAKE MOMENT         SINDH     KARACHI       ZONE-C   
1     CHOC FUDGE BROWNI MOMENT         SINDH     KARACHI       ZONE-F   
2         KING CONE 2/1 (1X25)         SINDH     KARACHI  LOWER SINDH   
3     MANGO CLASSIC CUP (1X16)         SINDH     KARACHI  LOWER SINDH   
4      CHEER VANILA CUP (1X24)         SINDH     KARACHI  LOWER SINDH   

                StoreName       StoreArea        Route  PaymentTerms  \
0         KOILA RESTURANT  C.P BARAR Town  O

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46258 entries, 0 to 46257
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SalesDate           46258 non-null  datetime64[ns]
 1   SalesQty            46258 non-null  int64         
 2   SalesAmount         46257 non-null  float64       
 3   ProductCategory     46258 non-null  object        
 4   ProductSubCategory  46258 non-null  object        
 5   ProductName         46258 non-null  object        
 6   StoreProvince       46258 non-null  object        
 7   StoreRegion         46257 non-null  object        
 8   StoreZone           46258 non-null  object        
 9   StoreName           46254 non-null  object        
 10  StoreArea           46191 non-null  object        
 11  Route               46241 non-null  object        
 12  PaymentTerms        46258 non-null  object        
 13  SalesMan            46258 non-null  object    

In [ ]:
schema=StructType(
    [StructField('SalesDate',DateType(),True),
     StructField('SalesQty',IntegerType(),True),
     StructField('SalesAmount',FloatType(),True),
     StructField('ProductCategory',StringType(),True),
      StructField('ProductSubCategory',StringType(),True),
     StructField('ProductName',StringType(),True),
     StructField('StoreProvince',StringType(),True),
     StructField('StoreRegion',StringType(),True),
     StructField('StoreZone',StringType(),True),
     StructField('StoreName',StringType(),True),
     StructField('StoreArea',StringType(),True),
     StructField('Route',StringType(),True),
     StructField('PaymentTerms',StringType(),True),
     StructField('SalesMan',StringType(),True),
     StructField('Category',StringType(),True)
]
)

In [ ]:
df.columns

Index(['SalesDate', 'SalesQty', 'SalesAmount', 'ProductCategory',
       'ProductSubCategory', 'ProductName', 'StoreProvince', 'StoreRegion',
       'StoreZone', 'StoreName', 'StoreArea', 'Route', 'PaymentTerms',
       'SalesMan', 'Category'],
      dtype='object')

In [ ]:
df2=spark.createDataFrame(df,schema=schema)

In [ ]:
df2.dtypes

[('SalesDate', 'date'),
 ('SalesQty', 'int'),
 ('SalesAmount', 'float'),
 ('ProductCategory', 'string'),
 ('ProductSubCategory', 'string'),
 ('ProductName', 'string'),
 ('StoreProvince', 'string'),
 ('StoreRegion', 'string'),
 ('StoreZone', 'string'),
 ('StoreName', 'string'),
 ('StoreArea', 'string'),
 ('Route', 'string'),
 ('PaymentTerms', 'string'),
 ('SalesMan', 'string'),
 ('Category', 'string')]

In [ ]:
#Creating Temp tables from df2
df2.createOrReplaceTempView('dim_product')
spark.sql('select ProductCategory,ProductSubCategory,ProductName from dim_product')

DataFrame[ProductCategory: string, ProductSubCategory: string, ProductName: string]

In [ ]:
dimproduct=spark.sql('select ProductCategory,ProductSubCategory,ProductName from dim_product').distinct()

In [ ]:
dimproduct.show()

+--------------------+------------------+--------------------+
|     ProductCategory|ProductSubCategory|         ProductName|
+--------------------+------------------+--------------------+
|               STICK|   STICK ICE LOLLY|      MAMAMIA (1X50)|
|          Blue Bell |         BALL CONE|    BALL CONE (1X20)|
|          Chapman's |        FUNDAY CUP|FUNDE CHOCLATE CU...|
| Casper's Ice Cream |           SUPREME|MANGO RIPPLE SUPREME|
|      Cows Creamery |       1 LITRE SPL|SHAHI KULFA PARTY...|
|            Breyers |   STICK ICE LOLLY|      MAMAMIA (1X50)|
|     Double Rainbow |    HALF LITRE ODR|STRAWBERRY FAMILY...|
|            Breyers |           2 LITRE|     TRIPPLE DELIGHT|
|          Blue Bell |       1 LITRE ODR|     ROSE PARTY PACK|
|          Creambell |        STICK LAVA|LAVA CHOCLATE (1X30)|
|            Braum's |           2 LITRE|     TRIPPLE DELIGHT|
|          D'Onofrio |        FUNDAY CUP|DIET STRABERRY CU...|
|Bonnie Doon Ice C...|   STICK ICE LOLLY|ICE CREAM SODA

In [ ]:
dimproduct=dimproduct.withColumn('productID',row_number().over(Window.orderBy(monotonically_increasing_id())))

In [ ]:
dimproduct.show()

+--------------------+------------------+--------------------+---------+
|     ProductCategory|ProductSubCategory|         ProductName|productID|
+--------------------+------------------+--------------------+---------+
|               STICK|   STICK ICE LOLLY|      MAMAMIA (1X50)|        1|
|          Blue Bell |         BALL CONE|    BALL CONE (1X20)|        2|
|          Chapman's |        FUNDAY CUP|FUNDE CHOCLATE CU...|        3|
| Casper's Ice Cream |           SUPREME|MANGO RIPPLE SUPREME|        4|
|      Cows Creamery |       1 LITRE SPL|SHAHI KULFA PARTY...|        5|
|            Breyers |   STICK ICE LOLLY|      MAMAMIA (1X50)|        6|
|     Double Rainbow |    HALF LITRE ODR|STRAWBERRY FAMILY...|        7|
|            Breyers |           2 LITRE|     TRIPPLE DELIGHT|        8|
|          Blue Bell |       1 LITRE ODR|     ROSE PARTY PACK|        9|
|          Creambell |        STICK LAVA|LAVA CHOCLATE (1X30)|       10|
|            Braum's |           2 LITRE|     TRIPP

In [ ]:
dim_Product=dimproduct.toPandas()
dim_Product.to_excel('/content/dim_product.xlsx',header=True,index=False)

In [ ]:
df2.createOrReplaceTempView('dim_store')
dimstore=spark.sql('select StoreProvince,StoreRegion,StoreZone, StoreName, StoreArea from dim_store').distinct()


In [ ]:
dimstore=dimstore.withColumn('StoreID',row_number().over(Window.orderBy(monotonically_increasing_id())))

In [ ]:
dim_store=dimstore.toPandas()
dim_store=dim_store.reindex(columns=['StoreName','StoreProvince','StoreRegion','StoreZone', 'StoreArea','StoreID'])
dim_store.to_excel('/content/dim_store.xlsx',header=True,index=False)

In [ ]:
dimstore.show()

+-------------+-----------+-----------+--------------------+--------------------+-------+
|StoreProvince|StoreRegion|  StoreZone|           StoreName|           StoreArea|StoreID|
+-------------+-----------+-----------+--------------------+--------------------+-------+
|        SINDH|    KARACHI|     ZONE-E|          BEST FOODS|         LANDHI NO 2|      1|
|        SINDH|  HYDERABAD|     ZONE-C|      GHYMKHANA REST|             SANGHAR|      2|
|        SINDH|  HYDERABAD|     ZONE-I| HASSANS SUPER STORE|     LATIFABAD NO 10|      3|
|        SINDH|  HYDERABAD|     ZONE-I|     BISMILLAH SUPER|      LATIFABAD NO 3|      4|
|        SINDH|  HYDERABAD|LOWER SINDH|          BABA FARID|             JHATPAT|      5|
|        SINDH|  HYDERABAD|LOWER SINDH|   FRIENDS RESTURANT|         MIRPUR KHAS|      6|
|        SINDH|    KARACHI|     ZONE-D|          C-I-P LAWN|  P.I.A STAFF COLONY|      7|
|        SINDH|    KARACHI|     ZONE-F|  MEHRAN SUPER STORE|SECTOR 4-D SAEED ...|      8|
|        S

In [ ]:
df2.createOrReplaceTempView('dim_dealer')
dimdealer=spark.sql('select SalesMan,Route from dim_dealer').distinct()
dimdealer=dimdealer.withColumn('DealerID',row_number().over(Window.orderBy(monotonically_increasing_id())))

In [ ]:
dimdealer.show()

+----------------+---------------+--------+
|        SalesMan|          Route|DealerID|
+----------------+---------------+--------+
|           MIRAL|      CHAKIWARA|       1|
| HADI ALI JAFRI |           MORO|       2|
|   FACTORY SALES|   FACTORY SALE|       3|
| HADI ALI JAFRI |      JACOBABAD|       4|
|        ADBULLAH|      LANDHI-25|       5|
|    GHULAM ABBAS|  NORTH KARACHI|       6|
|AFZAL IZZAT KHAN|GULSHAN-E-IQBAL|       7|
|     AHMED AMIN |  NORTH KARACHI|       8|
|AFZAL IZZAT KHAN|      LANDHI-25|       9|
|     AIJAZ AHMED| S.P.D ROUTE 02|      10|
|  HADI A JAFFRY |         THATTA|      11|
|      AMAN AFZAL|     KORANGI-28|      12|
|      AMAN AFZAL|      NAZIMABAD|      13|
|        ADBULLAH|         THATTA|      14|
|  FAIZAN JAMEEL |  NORTH KARACHI|      15|
|  FAIZAN JAMEEL |    NEW KARACHI|      16|
| HADI ALI JAFRI |           DADO|      17|
|  HADI A JAFFRY |           MORO|      18|
|     AHMED AMIN |       DASTAGIR|      19|
|     FURQAN KHAN|  NORTH KARACH

In [ ]:
dim_dealer=dimdealer.toPandas()
dim_dealer=dim_dealer.reindex(columns=['DealerID','SalesMan','Route'])

In [ ]:
dim_dealer.to_excel('/content/dim_dealer.xlsx',header=True,index=False)

In [ ]:
def create_date(start='2010-01-01',end='2050-12-21'):
  df=pd.DataFrame({"date":pd.date_range(start,end)})
  df["day"]=df.date.dt.day
  df["month"]=df.date.dt.month
  df["week"]=df.date.dt.isocalendar().week
  df["quarter"]=df.date.dt.quarter
  df["year"]=df.date.dt.year
  df.insert(0,'date_id',(df.year.astype(str)+
                       df.month.astype(str).str.zfill(2)+
                       df.day.astype(str).str.zfill(2)))
  return df

In [ ]:
dim_date=create_date()
dim_date.to_excel('/content/dim_date.xlsx',header=True,index=False)

In [ ]:
df2.createOrReplaceTempView("lkp_payment")
lkp=spark.sql('select PaymentTerms from lkp_payment').distinct()
lkp=lkp.withColumn('PaymentID',row_number().over(Window.orderBy(monotonically_increasing_id())))


In [ ]:
lkp_Payment=lkp.toPandas()
lkp_Payment.to_excel('/content/lkp_payment.xlsx',header=True,index=False)

In [ ]:
dimp=spark.createDataFrame(dim_Product)
dimp.createOrReplaceTempView('dimp')

In [ ]:
dimst=spark.createDataFrame(dim_store)
dimst.createOrReplaceTempView('dimst')


In [ ]:
dimd=spark.createDataFrame(dim_dealer)
dimd.createOrReplaceTempView('dimd')

In [ ]:
lkp=spark.createDataFrame(lkp_Payment)
lkp.createOrReplaceTempView('lkp')

In [ ]:
dk=spark.createDataFrame(dim_date)
dk.createOrReplaceTempView('dk')

In [ ]:
df2.createOrReplaceTempView('sales')

In [ ]:
print(dimst.columns)

['StoreName', 'StoreProvince', 'StoreRegion', 'StoreZone', 'StoreArea', 'StoreID']


In [ ]:
fact=spark.sql('select s.SalesQTY,s.SalesAmount,dp.productID,ds.StoreID,dd.DealerID,lkp.PaymentID,dk.date_id from sales s inner join dimp dp on s.ProductCategory=dp.ProductCategory and s.ProductSubCategory=dp.ProductSubCategory and s.ProductName=dp.ProductName inner join dimst ds on s.StoreName=ds.StoreName and s.StoreRegion=ds.StoreRegion and s.StoreZone=ds.StoreZone inner join dimd dd on s.SalesMan=dd.SalesMan and s.Route=dd.Route inner join lkp on s. PaymentTerms=lkp.PaymentTerms inner join dk on s.SalesDate=dk.date')

In [ ]:
fact_sales=fact.toPandas()

In [ ]:
fact_sales.to_excel('/content/fact_sales.xlsx',header=True,index=False)

In [ ]:
fact_sales.head()

,SalesQTY,SalesAmount,productID,StoreID,DealerID,PaymentID,date_id
0,50,26900.0,1,3379,59,2,20221119
1,75,40350.0,1,6881,59,2,20220315
2,50,26900.0,1,8281,59,2,20221220
3,50,26900.0,1,7417,59,2,20221220
4,50,26900.0,1,7100,59,2,20221220


In [ ]:
fact_sales['SalesAmount'].fillna(0).sum()


np.float32(10635319000.0)

In [ ]:
dim_dealer.head()

,DealerID,SalesMan,Route
0,1,MIRAL,CHAKIWARA
1,2,HADI ALI JAFRI,MORO
2,3,FACTORY SALES,FACTORY SALE
3,4,HADI ALI JAFRI,JACOBABAD
4,5,ADBULLAH,LANDHI-25


In [ ]:
dim_Product.head()

,ProductCategory,ProductSubCategory,ProductName,productID
0,STICK,STICK ICE LOLLY,MAMAMIA (1X50),1
1,Blue Bell,BALL CONE,BALL CONE (1X20),2
2,Chapman's,FUNDAY CUP,FUNDE CHOCLATE CUP (1X16),3
3,Casper's Ice Cream,SUPREME,MANGO RIPPLE SUPREME,4
4,Cows Creamery,1 LITRE SPL,SHAHI KULFA PARTY PACK,5


In [ ]:
dim_store.head()

,StoreName,StoreProvince,StoreRegion,StoreZone,StoreArea,StoreID
0,BEST FOODS,SINDH,KARACHI,ZONE-E,LANDHI NO 2,1
1,GHYMKHANA REST,SINDH,HYDERABAD,ZONE-C,SANGHAR,2
2,HASSANS SUPER STORE,SINDH,HYDERABAD,ZONE-I,LATIFABAD NO 10,3
3,BISMILLAH SUPER,SINDH,HYDERABAD,ZONE-I,LATIFABAD NO 3,4
4,BABA FARID,SINDH,HYDERABAD,LOWER SINDH,JHATPAT,5


In [ ]:
print(df2.show())

+----------+--------+-----------+--------------------+------------------+--------------------+-------------+-----------+-----------+--------------------+--------------------+-----------+------------+--------+--------+
| SalesDate|SalesQty|SalesAmount|     ProductCategory|ProductSubCategory|         ProductName|StoreProvince|StoreRegion|  StoreZone|           StoreName|           StoreArea|      Route|PaymentTerms|SalesMan|Category|
+----------+--------+-----------+--------------------+------------------+--------------------+-------------+-----------+-----------+--------------------+--------------------+-----------+------------+--------+--------+
|2022-09-02|       2|    36956.0|          ICE CREAMS|        MOMENT TUB|STRABERY CHEESE C...|        SINDH|    KARACHI|     ZONE-C|     KOILA RESTURANT|      C.P BARAR Town|ORANGI TOWN|Credit (114)|ADBULLAH|     RSO|
|2022-09-05|       5|    92390.0|          ICE CREAMS|        MOMENT TUB|CHOC FUDGE BROWNI...|        SINDH|    KARACHI|     ZON

In [ ]:
lkp_Payment.head()

,PaymentTerms,PaymentID
0,Credit (114),1
1,Cash,2
2,With in 30 Days(110),3


In [ ]:
print(sum(fact_sales['SalesAmount']))

nan


In [ ]:
dim_date['year']

,year
0,2010
1,2010
2,2010
3,2010
4,2010
...,...
14960,2050
14961,2050
14962,2050
14963,2050


In [ ]:
dim_Product.head()

,ProductCategory,ProductSubCategory,ProductName,productID
0,STICK,STICK ICE LOLLY,MAMAMIA (1X50),1
1,Blue Bell,BALL CONE,BALL CONE (1X20),2
2,Chapman's,FUNDAY CUP,FUNDE CHOCLATE CUP (1X16),3
3,Casper's Ice Cream,SUPREME,MANGO RIPPLE SUPREME,4
4,Cows Creamery,1 LITRE SPL,SHAHI KULFA PARTY PACK,5
